# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
vacation_data = pd.read_csv("weatherpy_data.csv") 
vacation_df=pd.DataFrame(vacation_data)
vacation_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,0,longyearbyen,75,SJ,1603849784,86,78.22,15.64,30.20,2.24
1,1,chokurdakh,100,RU,1603849955,97,70.63,147.92,12.97,4.65
2,2,cape town,75,ZA,1603849760,87,-33.93,18.42,61.00,10.29
3,3,khvastovichi,0,RU,1603850200,80,53.47,35.09,42.28,8.21
4,4,east london,50,ZA,1603849765,75,-33.02,27.91,66.87,16.22
...,...,...,...,...,...,...,...,...,...,...
580,580,quatre cocos,16,MU,1603849628,80,-20.21,57.76,70.00,4.00
581,581,mehamn,100,NO,1603849749,83,71.04,27.85,36.05,9.40
582,582,ferkessedougou,31,CI,1603850354,57,9.60,-5.20,75.11,2.48
583,583,vostochnyy,0,RU,1603850354,87,55.82,37.87,45.00,8.95


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(api_key=g_key)
locations=vacation_df[["Lat","Lng"]]
humidity_weight=vacation_df["Humidity"].astype(float)

In [15]:
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations,weights=humidity_weight,dissipating=False, max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
new_weather_df = pd.DataFrame(vacation_df, columns = ["City", "Max Temperature", "Wind Speed", "Cloudiness","Lat", "Lng", "Country"])



criteria = new_weather_df.loc[(new_weather_df["Max Temperature"] <= 80) & (new_weather_df["Max Temperature"] > 70) & (new_weather_df["Wind Speed"] < 10)& (new_weather_df["Cloudiness"] == 0)]
#wind_speed = new_weather_df.loc[new_weather_df["Wind Speed"] < 10]
#cloudiness = new_weather_df["Cloudiness"] == 0
criteria
#new_weather_df
#type(max_temp)
#new_weather_df[max_temp, wind_speed, cloudiness]

,City,Max Temperature,Wind Speed,Cloudiness,Lat,Lng,Country
69,tahoua,75.20,5.84,0,14.89,5.27,NE
118,eyl,75.74,4.83,0,7.98,49.82,SO
125,kutum,70.56,8.48,0,14.20,24.67,SD
173,ogaminana,73.78,3.20,0,7.59,6.22,NG
179,sao felix do xingu,75.58,2.28,0,-6.64,-51.99,BR
184,oussouye,79.32,4.72,0,12.48,-16.55,SN
276,teofilo otoni,72.03,2.82,0,-17.86,-41.51,BR
306,tera,77.70,3.91,0,14.01,0.75,NE
307,tura,75.13,3.15,0,25.52,90.22,IN
309,hwange,72.36,4.45,0,-18.37,26.50,ZW


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
hotel_df = pd.DataFrame(criteria, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

hotel_df




,City,Country,Lat,Lng,Hotel Name
69,tahoua,NE,14.89,5.27,
118,eyl,SO,7.98,49.82,
125,kutum,SD,14.20,24.67,
173,ogaminana,NG,7.59,6.22,
179,sao felix do xingu,BR,-6.64,-51.99,
184,oussouye,SN,12.48,-16.55,
276,teofilo otoni,BR,-17.86,-41.51,
306,tera,NE,14.01,0.75,
307,tura,IN,25.52,90.22,
309,hwange,ZW,-18.37,26.50,


In [57]:
#target_coordinates = f"{hotel_df['Lat']},{hotel_df['Lng']}"
#target_search = "hotel"
#target_radius = 5000
#target_type = "hotel"
hotel_list=[]


for i, row in hotel_df.iterrows():  
    
    params = {
    "location": "{0},{1}".format(row['Lat'],row['Lng']),
    "keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    response = requests.get(base_url, params = params).json()

    try:
        hotel_list.append(response['results'][0]['name'])
   
    except IndexError:
        print('The city ' + str(row['City']) + 'has no hotels within 5000m')
        hotel_list.append('')

hotel_df['Hotel Name'] = hotel_list


The city tahouahas no hotels within 5000m
The city eylhas no hotels within 5000m
The city kutumhas no hotels within 5000m
The city ogaminanahas no hotels within 5000m
The city sao felix do xinguhas no hotels within 5000m
The city oussouyehas no hotels within 5000m
The city teofilo otonihas no hotels within 5000m
The city terahas no hotels within 5000m
The city turahas no hotels within 5000m
The city hwangehas no hotels within 5000m
The city kumlucahas no hotels within 5000m
The city namibehas no hotels within 5000m
The city toumodihas no hotels within 5000m
The city general salgadohas no hotels within 5000m
The city ankazoabohas no hotels within 5000m
The city manganhas no hotels within 5000m
The city murray bridgehas no hotels within 5000m
The city tessalithas no hotels within 5000m
The city shakawehas no hotels within 5000m
The city birnin kebbihas no hotels within 5000m


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [59]:
hotel_list

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [60]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig


# Display figure


Figure(layout=FigureLayout(height='420px'))